In [4]:
import pandas as pd

def get_semester_dates(year, semester):
    """
    Returns the start and end date of the semester based on the given year and semester type, respecting UTC time.
    Semesters:
    - Spring: January to June
    - Summer: June to July
    - Winter: August to December
    """
    if semester == 'Spring':
        return pd.Timestamp(f'{year}-01-01', tz='UTC'), pd.Timestamp(f'{year}-06-30', tz='UTC')
    elif semester == 'Summer':
        return pd.Timestamp(f'{year}-06-01', tz='UTC'), pd.Timestamp(f'{year}-07-31', tz='UTC')
    elif semester == 'Winter':
        return pd.Timestamp(f'{year}-08-01', tz='UTC'), pd.Timestamp(f'{year}-12-31', tz='UTC')
    else:
        raise ValueError("Invalid semester. Choose between 'Spring', 'Summer', or 'Winter'.")


In [7]:
def create_course_availability_and_activity_for_semester(courses, year, semester):
    # Get the start and end date for the selected semester with UTC time
    start_semester, end_semester = get_semester_dates(year, semester)

    # Convert course dates to UTC
    courses['start_at'] = pd.to_datetime(courses['start_at'], utc=True)
    courses['end_at'] = pd.to_datetime(courses['end_at'], utc=True)

    # Filter active courses within the semester
    active_courses = courses[
        (courses['workflow_state'] == 'active') &
        (courses['start_at'] <= end_semester) &
        ((courses['end_at'] >= start_semester) | courses['end_at'].isna())
    ]

    # Filter inactive courses within the semester
    inactive_courses = courses[
        (courses['workflow_state'] != 'active') &
        (courses['start_at'] <= end_semester) &
        (courses['end_at'] < start_semester) & 
        courses['end_at'].notna()  # Ensure the course has ended
    ]

    # Calculate counts
    active_count = active_courses.shape[0]
    inactive_count = inactive_courses.shape[0]

    # Calculate ratio of active to inactive courses
    if inactive_count > 0:
        ratio_active_to_inactive = active_count / inactive_count
    else:
        ratio_active_to_inactive = float('inf')

    return active_count, inactive_count, ratio_active_to_inactive
